In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
import peakutils
from sklearn.linear_model import LogisticRegression

#from mpl_toolkits.mplot3d import Axes3D

sensorData = []
soundData  = []
labels = []

punch0 = []
punch1 = []
punch2 = []
punch3 = []
punch4 = []


 
with open ('sensors1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        line = []
        for x in row.split(','):
            line1 = []
            for y in x.split(' '):
                try:
                    line1.append(float(y))
                except Exception:
                    pass
            if len(line1) == 6:
                line.append(line1)
        if len(line) == 100:
            sensorData.append(line)
with open ('sounds1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        empty = []
        for i in row.split(','):
            empty.append(float(i))
        soundData.append(empty)
with open ('labels1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        labels.append(int(row))

for x in range(len(sensorData)):
    if x % 5 == 0:
        punch0.append(sensorData[x])
    if x % 5 == 1:
        punch1.append(sensorData[x])
    if x % 5 == 2:
        punch2.append(sensorData[x])
    if x % 5 == 3:
        punch3.append(sensorData[x])
    if x % 5 == 4:
        punch4.append(sensorData[x])
soundData  = np.array(soundData)
sensorData = np.array(sensorData)
labels     = np.array(labels)
punch0 = np.array(punch0)
punch1 = np.array(punch1)
punch2 = np.array(punch2)
punch3 = np.array(punch3)
punch4 = np.array(punch4)

print(soundData.shape)


(550, 400)


In [2]:
from scipy.fftpack import fft, rfft
sensorData = np.array(sensorData)

def zeroCrossings(x):
    zero_crossings = np.where(np.diff(np.sign(x)))[0]
    return(len(zero_crossings))
    

def featureExtraction(x, sound):
    features = []
    for u in range(6):
        features.append(np.mean(x[:100, u]))
        features.append(np.max(x[:100, u]))
        features.append(np.min(x[:100, u]))
        features.append(np.std(x[:100, u]))
        features.append(zeroCrossings(x[:100, u]))
        features.append(np.var(x[:100, u]))
    features.append(np.std(x[:100, :3]))
    features.append(np.std(x[:100, 3:]))
    a, b, c, d, e, f, g, h, i = topPeaks(sound)
    features.append(a)
    features.append(b)
    features.append(c)
    features.append(d)
    features.append(e)
    features.append(f)
    features.append(g)
    features.append(h)
    features.append(i)
    return features

def topPeaks(sound):
    fourier = abs(rfft(sound))
    phase = (np.angle(rfft(sound)))
    indices = peakutils.indexes(fourier, thres=0.02/max(fourier), min_dist=0.1)
    amplitudes = (fourier[indices])
    amplitudes = (np.sort(amplitudes))[len(amplitudes)-3:len(amplitudes)]
    a    = fourier.tolist().index(amplitudes[0])
    b    = fourier.tolist().index(amplitudes[1])
    c    = fourier.tolist().index(amplitudes[2])
    d, e, f = amplitudes
    g    = phase[a]
    h    = phase[b]
    i    = phase[c]
    return a, b, c, d, e, f, g, h, i

topPeaks(soundData[250,0:])
# for x in range(len(sensorData)):
#     print(x)
#     print(featureExtraction(sensorData[x], soundData[x, 0:]))  
features = []

for x in range(len(sensorData)):
     features.append(featureExtraction(sensorData[x], soundData[x, 0:]))

features = np.array(features)

training = features[100:]
test     = features[:50]
validation = features[50:100]
training_labels = labels[100:]
test_labels = labels[:50]
validaton_labels = labels[50:100]

print(validation.shape)
print(training.shape)
print(test.shape)

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels  = labels[permutation]
    return shuffled_dataset, shuffled_labels


train_dataset, train_labels = randomize(training, training_labels)
test_dataset, test_labels = randomize(test, test_labels)
validation_dataset, validation_labels = randomize(validation, validaton_labels)



(50, 47)
(450, 47)
(50, 47)


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from tensorflow.contrib import learn
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from numpy import reciprocal

In [4]:
# params = {
#     'kernel': ['poly', 'linear', 'rbf', 'sigmoid'],
# }

# clf = SVC()
# rnd_search = GridSearchCV(clf, param_grid=params)
# rnd_search.fit(train_dataset,train_labels)


In [5]:
# rnd_search.best_params_

In [6]:
# classifier = LinearSVC(random_state=42, penalty='l1', max_iter=40000)
# classifier.fit(train_dataset, train_labels)

In [7]:
# classifier.score(test_dataset, test_labels)

In [8]:
# params1 = {
#     'hidden_layer_sizes': [(27,3), (28,3), (29,3), (27, 2), (28, 2), (29, 2)],
#     'activation': ['logistic', 'tanh', 'relu'],
#     'solver': ['lbfgs', 'sgd'],
#     'max_iter': [5000, 6000, 7000],
#     'random_state': [42],
#     'learning_rate': ['constant', 'adaptive'],
#     'learning_rate_init': [0.1]
# }
# clf1 = MLPClassifier()
# rnd_search1 = GridSearchCV(clf1, param_grid=params1)
# rnd_search1.fit(train_dataset,train_labels)

In [9]:
# rnd_search1.best_score_
# rnd_search1.best_estimator_


In [10]:
# classifier1 =MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
#        beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
#        hidden_layer_sizes=(29, 3), learning_rate='constant',
#        learning_rate_init=0.1, max_iter=5000, momentum=0.9,
#        nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
#        solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
#        warm_start=False)
# classifier1.fit(train_dataset, train_labels)
# classifier1.score(train_dataset, train_labels)


In [11]:
# for i in range(1, 300):
#     classifier1 = MLPClassifier(hidden_layer_sizes= (i, ), solver='sgd', random_state=42, max_iter=100000, learning_rate= 'constant', activation='logistic', learning_rate_init=0.01) 
#     classifier1.fit(train_dataset, train_labels)
#     if classifier1.score(test_dataset, test_labels) > 0.6:
#         print(i)

    


In [12]:

feature_columns = learn.infer_real_valued_columns_from_input(train_dataset)
classifier = learn.LinearClassifier(n_classes = 5, feature_columns=feature_columns,
                                    optimizer=tf.train.FtrlOptimizer(
                                    learning_rate_power=-0.69,
                                    learning_rate=0.00001,
                                    
                                    l2_regularization_strength=0.1))
#classifier = learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10, 20, 10], n_classes=5)
classifier.fit(train_dataset, train_labels, steps=30000)
# classifier.fit(train_dataset, train_labels)





Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9d36e6a978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local

INFO:tensorflow:loss = 0.5841139, step = 1500 (0.141 sec)
INFO:tensorflow:global_step/sec: 707.175
INFO:tensorflow:loss = 0.5800597, step = 1600 (0.141 sec)
INFO:tensorflow:global_step/sec: 724.232
INFO:tensorflow:loss = 0.57623726, step = 1700 (0.138 sec)
INFO:tensorflow:global_step/sec: 723.825
INFO:tensorflow:loss = 0.57262206, step = 1800 (0.139 sec)
INFO:tensorflow:global_step/sec: 708.698
INFO:tensorflow:loss = 0.56919307, step = 1900 (0.140 sec)
INFO:tensorflow:global_step/sec: 715.914
INFO:tensorflow:loss = 0.56593233, step = 2000 (0.140 sec)
INFO:tensorflow:global_step/sec: 717.87
INFO:tensorflow:loss = 0.56282485, step = 2100 (0.139 sec)
INFO:tensorflow:global_step/sec: 728.457
INFO:tensorflow:loss = 0.55985695, step = 2200 (0.137 sec)
INFO:tensorflow:global_step/sec: 711.116
INFO:tensorflow:loss = 0.55701727, step = 2300 (0.141 sec)
INFO:tensorflow:global_step/sec: 714.116
INFO:tensorflow:loss = 0.5542956, step = 2400 (0.140 sec)
INFO:tensorflow:global_step/sec: 702.279
INFO

INFO:tensorflow:global_step/sec: 731.63
INFO:tensorflow:loss = 0.46733546, step = 9800 (0.137 sec)
INFO:tensorflow:global_step/sec: 723.273
INFO:tensorflow:loss = 0.46675956, step = 9900 (0.138 sec)
INFO:tensorflow:global_step/sec: 734.837
INFO:tensorflow:loss = 0.46619055, step = 10000 (0.136 sec)
INFO:tensorflow:global_step/sec: 727.643
INFO:tensorflow:loss = 0.4656281, step = 10100 (0.137 sec)
INFO:tensorflow:global_step/sec: 722.047
INFO:tensorflow:loss = 0.4650722, step = 10200 (0.138 sec)
INFO:tensorflow:global_step/sec: 717.159
INFO:tensorflow:loss = 0.46452275, step = 10300 (0.139 sec)
INFO:tensorflow:global_step/sec: 714.688
INFO:tensorflow:loss = 0.46397948, step = 10400 (0.140 sec)
INFO:tensorflow:global_step/sec: 719.042
INFO:tensorflow:loss = 0.4634423, step = 10500 (0.139 sec)
INFO:tensorflow:global_step/sec: 719.976
INFO:tensorflow:loss = 0.46291113, step = 10600 (0.139 sec)
INFO:tensorflow:global_step/sec: 741.271
INFO:tensorflow:loss = 0.46238586, step = 10700 (0.136 s

INFO:tensorflow:global_step/sec: 719.047
INFO:tensorflow:loss = 0.43463838, step = 18000 (0.139 sec)
INFO:tensorflow:global_step/sec: 727.701
INFO:tensorflow:loss = 0.43435663, step = 18100 (0.137 sec)
INFO:tensorflow:global_step/sec: 734.974
INFO:tensorflow:loss = 0.43407667, step = 18200 (0.136 sec)
INFO:tensorflow:global_step/sec: 728.942
INFO:tensorflow:loss = 0.43379855, step = 18300 (0.137 sec)
INFO:tensorflow:global_step/sec: 707.51
INFO:tensorflow:loss = 0.43352216, step = 18400 (0.141 sec)
INFO:tensorflow:global_step/sec: 701.745
INFO:tensorflow:loss = 0.43324766, step = 18500 (0.143 sec)
INFO:tensorflow:global_step/sec: 728.918
INFO:tensorflow:loss = 0.4329748, step = 18600 (0.137 sec)
INFO:tensorflow:global_step/sec: 636.635
INFO:tensorflow:loss = 0.43270364, step = 18700 (0.157 sec)
INFO:tensorflow:global_step/sec: 710.898
INFO:tensorflow:loss = 0.43243423, step = 18800 (0.141 sec)
INFO:tensorflow:global_step/sec: 722.178
INFO:tensorflow:loss = 0.43216646, step = 18900 (0.1

INFO:tensorflow:loss = 0.41632935, step = 26100 (0.137 sec)
INFO:tensorflow:global_step/sec: 701.033
INFO:tensorflow:loss = 0.416147, step = 26200 (0.143 sec)
INFO:tensorflow:global_step/sec: 723.626
INFO:tensorflow:loss = 0.41596535, step = 26300 (0.138 sec)
INFO:tensorflow:global_step/sec: 725.115
INFO:tensorflow:loss = 0.41578445, step = 26400 (0.138 sec)
INFO:tensorflow:global_step/sec: 719.772
INFO:tensorflow:loss = 0.41560447, step = 26500 (0.139 sec)
INFO:tensorflow:global_step/sec: 711.691
INFO:tensorflow:loss = 0.41542518, step = 26600 (0.140 sec)
INFO:tensorflow:global_step/sec: 725.671
INFO:tensorflow:loss = 0.4152467, step = 26700 (0.138 sec)
INFO:tensorflow:global_step/sec: 707.54
INFO:tensorflow:loss = 0.41506904, step = 26800 (0.141 sec)
INFO:tensorflow:global_step/sec: 702.552
INFO:tensorflow:loss = 0.414892, step = 26900 (0.142 sec)
INFO:tensorflow:global_step/sec: 730.244
INFO:tensorflow:loss = 0.41471586, step = 27000 (0.137 sec)
INFO:tensorflow:global_step/sec: 718.

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f9d36e6aba8>, 'feature_columns': [_RealValuedColumn(column_name='', dimension=47, default_value=None, dtype=tf.float64, normalizer=None)], 'optimizer': <tensorflow.python.training.ftrl.FtrlOptimizer object at 0x7f9d36e6a358>, 'gradient_clip_norm': None, 'joint_weights': False})

In [14]:
classifier.evaluate(test_dataset, test_labels)
# print(validation_labels)

# for i in classifier.predict(validation_dataset):
#     print(i)

# classifier.export(export_dir="./")

INFO:tensorflow:Starting evaluation at 2018-07-27-14:47:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp0j_sujh5/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-27-14:47:15
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.8, global_step = 30000, loss = 0.5777228


{'loss': 0.5777228, 'accuracy': 0.8, 'global_step': 30000}